# Experiment

## Imports

In [1]:
import argparse
from logging import getLogger

from Environments import (Algo, CartPole, Highway, Hopper, LunarLander,
                          Swimmer)
from LLM.LLMOptions import llm_options
from log.log_config import init_logger
from VIRAL import VIRAL
init_logger("DEBUG")

In [2]:
def runs(
    total_timesteps: int,
    nb_vec_envs: int,
    nb_refined: int,
    human_feedback: bool,
    video_description: bool,
    legacy_training: bool,
    actor_model: str,
    critic_model: str,
    env: str,
    observation_space: str,
    goal: str,
    image: str,
    nb_gen: int,
    nb_runs: int,
    proxies: dict,
    focus: str = "",
):
    """help wrapper for launch several runs

    Args:
        total_timesteps (int): 
        nb_vec_envs (int): 
        nb_refined (int): 
        human_feedback (bool): 
        video_description (bool): 
        legacy_training (bool): 
        actor_model (str): 
        critic_model (str): 
        env (str): 
        observation_space (str): 
        goal (str): 
        image (str): 
        nb_gen (int): 
        nb_runs (int): 
        proxies (dict): 
        focus (str, optional): . Defaults to "".
    """
    switcher = {
        "Cartpole": CartPole,
        "LunarLander": LunarLander,
        "Highway": Highway,
        "Swimmer": Swimmer,
        "Hopper": Hopper,
    }
    instance = switcher[env]()
    if observation_space != "":
        instance.prompt["Observation Space"] = observation_space
    if goal is not None:
        instance.prompt["Goal"] = goal
    else:
        instance.prompt.pop("Goal", None)
    if image is not None:
        instance.prompt["Image"] = image
    else:
        instance.prompt.pop("Image", None)
    def run():
        viral = VIRAL(
            env_type=instance,
            model_actor=actor_model,
            model_critic=critic_model,
            hf=human_feedback,
            vd=video_description,
            nb_vec_envs=nb_vec_envs,
            options=llm_options,
            legacy_training=legacy_training,
            training_time=total_timesteps,
            proxies=proxies,
        )
        viral.generate_context()
        viral.generate_reward_function(nb_gen, nb_refined, focus)
        viral.policy_trainer.start_vd(viral.memory[1].policy, 1)

    for r in range(nb_runs):
        print(f"#######  {r}  ########")
        run()

In [3]:
proxies = { 
	"http"  : "socks5h://localhost:1080", 
	"https" : "socks5h://localhost:1080", 
}

## LunarLander Compare Image w/o

In [ ]:
obs_space = """Box([ -2.5 -2.5 -10. -10. -6.2831855 -10. -0. -0. ], 
[ 2.5 2.5 10. 10. 6.2831855 10. 1. 1. ], (8,), float32)
The state is an 8-dimensional vector: 
the coordinates of the lander in x & y, 
its linear velocities in x & y, 
its angle, its angular velocity, 
and two booleans that represent whether each leg is in contact with the ground or not.
"""
goal = "Do not land but do not crash, i want a stationary Flight"
image = 'Environments/img/LunarLander_Stationary.png'

### Only text

In [9]:
runs(30_000, 1, 0, False, False, False, "qwen2.5-coder:32b", "llama3.2-vision", "LunarLander", obs_space, goal, None, 1, 10, proxies)


21:28:56 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Assuming Numpy already imported as np
        6. Take into the observation of the state, the is_success boolean flag, the is_failure boolean flag
        , Options: {'temperature': 0.9, 'seed': 356471}

21:28:56 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert, great mathematician and physician.
        You're an assistant in rewarding for the LunarLander-v3 environment.
        As a critic, you're going to explains step by step, how the agent can in theory achieve the goal: Don't land but don't crash, i want a stationary Flight.
     

#######  0  ########



21:29:04 OllamaChat.py:195 INFO
	Response -1:


<HELP>
The observation space consists of 8 dimensions, each with its own range of values:

1. `x` position of the lander (range: [-2.5, 2.5])
2. `y` position of the lander (range: [-2.5, 2.5])
3. `z` position of the lander (range: [-10, 10]) - Note that z is not explicitly mentioned in the problem description, but it's a part of the observation space.
4. `vx` linear velocity in x direction (range: [-10, 10])
5. `vy` linear velocity in y direction (range: [-10, 10])
6. `theta` angle of the lander (range: [-6.2831855, 6.2831855]) - Note that this value is actually a unit circle (2π) and represents the angle in radians.
7. `omega` angular velocity (range: [-1, 1])
8. Two booleans indicating whether each leg is in contact with the ground or not.

To achieve a stationary flight where the lander doesn't crash and doesn't land, the agent needs to control the following aspects:

* Keep the `z` position stable around 0
* Maintain a safe distance from the ground by keeping both legs off the grou


21:29:23 OllamaChat.py:195 INFO
	Response 0:


...


21:41:31 OllamaChat.py:195 INFO
	Response 0:


def reward_func(observations:np.ndarray, is_success:bool, is_failure:bool) -> float:
    x, y, dx, dy, angle, d_angle, leg1_contact, leg2_contact = observations

    altitude_reward = 10 if -0.5 <= y <= 0.5 else 0
    position_reward = 10 if -1 <= x <= 1 else 0
    angle_reward = 10 if abs(angle) < 0.1 else 0
    angular_velocity_reward = 10 if -1 <= d_angle <= 1 else 0

    crash_penalty = -100 if abs(x) > 2 or y < -1 else 0
    landing_penalty = -50 if 0 <= y < 0.1 else 0
    angle_deviation_penalty = -20 if abs(angle) > 0.5 else 0

    reward = altitude_reward + position_reward + angle_reward + angular_velocity_reward + crash_penalty + landing_penalty + angle_deviation_penalty

    return reward


21:41:43 GenCode.py:228 DEBUG
	Reward function output: 30

21:41:43 PolicyTrainer.py:60 INFO
	state 1 begin is learning

21:41:43 PolicyTrainer.py:319 DEBUG
	simple env


Output()


21:42:37 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

21:42:37 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/LunarLander-v3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


### Only Image

In [5]:
runs(30_000, 1, 0, False, False, False, "qwen2.5-coder:32b", "llama3.2-vision", "LunarLander", obs_space, None, image, 1, 10, proxies)


22:32:03 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Assuming Numpy already imported as np
        6. Take into the observation of the state, the is_success boolean flag, the is_failure boolean flag
        , Options: {'temperature': 0.9, 'seed': 207173}

22:32:03 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert, great mathematician and physician.
        As a critic, you're going to explains step by step, the environment.
        Every answer you make will be contained in an xml <HELP> tag., Options: {'temperature': 0.9, 'seed': 207173}


#######  0  ########



22:32:04 OllamaChat.py:195 INFO
	Response -1:


<HELP>

**Goal Annotation:**
The red annotation indicates "STAY IN THE BOX", which suggests that the goal of the agent is to remain within the designated boundaries or container.

**Goal Achievement:**
Based on this understanding, the primary objective for the agent is to successfully land and stay inside a box or a specific area, avoiding any collisions with its surroundings or falling outside the defined limits.

</HELP>

**Observation Space Description:**

The Observation Space consists of an 8-dimensional vector that represents various state variables essential for controlling the lander's movement and stability. These dimensions include:

1. **X-coordinate:** The agent's position along the horizontal axis (x-axis).
2. **Y-coordinate:** The agent's position along the vertical axis (y-axis).
3. **Linear Velocity X:** The speed at which the lander is moving horizontally.
4. **Linear Velocity Y:** The speed at which the lander is moving vertically.
5. **Angle:** The orientation of the


22:32:11 OllamaChat.py:195 INFO
	Response 0:


...

File "/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1736, in 
_wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

File "/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1747, in 
_call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

File "/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/torch/nn/modules/linear.py", line 125, in 
forward
    return F.linear(input, self.weight, self.bias)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt

KeyboardInterrupt: 

### Text+Image

In [ ]:
runs(30_000, 1, 0, False, False, False, "qwen2.5-coder:32b", "llama3.2-vision", "LunarLander", obs_space, goal, image, 1, 10, proxies)

## SwimmerCompare Image w/o

### with

In [4]:
obs_space = """Box(-inf, inf, (8,), float64)

The observation space consists of the following elements (in order):
- qpos (3 elements by default): Position values of the robot’s body parts.
- qvel (5 elements): Velocities of these body parts (their derivatives).

By default, the observation does not include the x- and y-coordinates of the front end. These can be included by passing `exclude_current_positions_from_observation=False` during construction. In this case, the observation space will be `Box(-Inf, Inf, (10,), float64)`, where the first two observations are the x- and y-coordinates of the front end. Regardless of the value of `exclude_current_positions_from_observation`, the x- and y-coordinates are returned in `info` with the keys "x_position" and "y_position", respectively.

By default, the observation space is `Box(-Inf, Inf, (8,), float64)` with the following elements:

| Num | Observation                                | Min  | Max  | Type                   |
|-----|--------------------------------------------|------|------|------------------------|
| 0   | Angle of the front end                    | -Inf | Inf  | angle (rad)            |
| 1   | Angle of the first joint                  | -Inf | Inf  | angle (rad)            |
| 2   | Angle of the second joint                 | -Inf | Inf  | angle (rad)            |
| 3   | Velocity of the front end along the x-axis| -Inf | Inf  | velocity (m/s)         |
| 4   | Velocity of the front end along the y-axis| -Inf | Inf  | velocity (m/s)         |
| 5   | Angular velocity of the front end         | -Inf | Inf  | angular velocity (rad/s) |
| 6   | Angular velocity of the first joint       | -Inf | Inf  | angular velocity (rad/s) |
| 7   | Angular velocity of the second joint      | -Inf | Inf  | angular velocity (rad/s) |"""

goal = "Control the swimmer to move as fast as possible in the forward direction."

img = "Environments/img/swimmer_forward.png"

runs(500_000, 2, 0, False, False, False, "qwen2.5-coder:32b", "llama3.2-vision", "Swimmer", obs_space, goal, img, 1, 100, proxies)


  50% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249,818/500,000  [ 0:02:01 < 0:01:59 , 2,106 it/s ]


19:52:12 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

19:52:12 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/Swimmer-v5 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init

19:52:19 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Ass

#######  44  ########



19:52:19 OllamaChat.py:195 INFO
	Response -1:


# HELP

The red trajectory represents the path that the swimmer has taken or is currently taking. This information can be used to infer various aspects of its movement and velocity.

**Observations Relevant to Goal:**

* The observation space consists of position values (`qpos`) and velocities (`qvel`) for the robot's body parts, excluding the x- and y-coordinates of the front end.
* Specifically, the relevant observations related to our goal include:
	+ Angle of the front end
	+ Velocity of the front end along the x-axis
	+ Angular velocity of the front end

**Red Trajectory Interpretation:**

The red trajectory can indicate:

* The direction in which the swimmer is currently moving or has been moving.
* Any deviations from a straight path, suggesting changes in speed or direction.

**Control Strategy:**

To control the swimmer to move as fast as possible in the forward direction:

1. **Use Angle Observations:** Adjust the angle of the front end to align with the desired direction (po

...


00:18:54 GenCode.py:99 WARNING
	Error syntax Syntax error in the generated code : 'return' outside function (<string>, line 10)

00:18:54 OllamaChat.py:195 INFO
	Response 1:


def reward_func(observations: np.ndarray, is_success: bool, is_failure: bool) -> float:
    """Reward function for Swimmer-v5"""
    x_velocity = observations[3]
    if is_success:
        return 100.0
    elif is_failure:
        return -10.0
    else:
        return x_velocity * 0.1


00:18:58 GenCode.py:228 DEBUG
	Reward function output: 0.0054193950604931405

00:18:58 PolicyTrainer.py:60 INFO
	state 1 begin is learning


Output()


00:23:27 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.02

00:23:28 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/Swimmer-v5 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init


### without

In [6]:
obs_space = """Box(-inf, inf, (8,), float64)

The observation space consists of the following elements (in order):
- qpos (3 elements by default): Position values of the robot’s body parts.
- qvel (5 elements): Velocities of these body parts (their derivatives).

By default, the observation does not include the x- and y-coordinates of the front end. These can be included by passing `exclude_current_positions_from_observation=False` during construction. In this case, the observation space will be `Box(-Inf, Inf, (10,), float64)`, where the first two observations are the x- and y-coordinates of the front end. Regardless of the value of `exclude_current_positions_from_observation`, the x- and y-coordinates are returned in `info` with the keys "x_position" and "y_position", respectively.

By default, the observation space is `Box(-Inf, Inf, (8,), float64)` with the following elements:

| Num | Observation                                | Min  | Max  | Type                   |
|-----|--------------------------------------------|------|------|------------------------|
| 0   | Angle of the front end                    | -Inf | Inf  | angle (rad)            |
| 1   | Angle of the first joint                  | -Inf | Inf  | angle (rad)            |
| 2   | Angle of the second joint                 | -Inf | Inf  | angle (rad)            |
| 3   | Velocity of the front end along the x-axis| -Inf | Inf  | velocity (m/s)         |
| 4   | Velocity of the front end along the y-axis| -Inf | Inf  | velocity (m/s)         |
| 5   | Angular velocity of the front end         | -Inf | Inf  | angular velocity (rad/s) |
| 6   | Angular velocity of the first joint       | -Inf | Inf  | angular velocity (rad/s) |
| 7   | Angular velocity of the second joint      | -Inf | Inf  | angular velocity (rad/s) |"""

goal = "Control the swimmer to move as fast as possible in the forward direction."

img = ""

runs(500_000, 2, 0, False, False, False, "qwen2.5-coder:32b", "llama3.2-vision", "Swimmer", obs_space, goal, img, 1, 100, proxies)


  26% ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130,748/500,000  [ 0:01:02 < 0:02:57 , 2,095 it/s ]


11:51:15 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 1.0

11:51:15 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/Swimmer-v5 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init

11:51:22 OllamaChat.py:39 INFO
	System: 
        You're a reinforcement learning expert specializing in the design of python reward functions.
        Strict criteria:
        1. Take care of Generate ALWAYS DIFFERENTS reward function per Response iteration
        2. Complete ONLY the reward function code
        3. Give no additional explanations
        4. STOP immediately your completion after the last return
        5. Ass

#######  54  ########
# HELP
To describe the red trajectory and observations for the goal of controlling the swimmer to move as fast as possible in the forward direction, I'll analyze each observation space element:

- **Angle of the front end (0)**: This value represents the orientation of the front end of the robot. To achieve maximum speed in the forward direction, the angle should be aligned with the x-axis, ideally at 0 radians.

- **Angle of the first joint (1)**: Similar to the front end's angle, this should also be aligned with the x-axis for efficient movement. A value close to 0 radians would ensure that the robot's body is not restricting its forward motion.

- **Angle of the second joint (2)**: Similarly, the second joint's angle should also align with the x-axis for optimal performance. Again, a value around 0 radians ensures proper alignment and no obstruction in the forward direction.

- **Velocity of the front end along the x-axis (3)**: This is directly related to our 


11:51:31 OllamaChat.py:195 INFO
	Response 0:


def reward_func(observations: np.ndarray, is_success: bool, is_failure: bool) -> float:
    x_velocity = observations[3]
    y_velocity = observations[4]
    front_end_angle = abs(observations[0])
    joint1_angle = abs(observations[1])
    joint2_angle = abs(observations[2])

    if is_success:
        return 50.0
    elif is_failure:
        return -50.0

    reward = x_velocity - 0.1 * (front_end_angle + joint1_angle + joint2_angle) - 0.1 * abs(y_velocity)
    return reward

...


15:24:21 OllamaChat.py:195 INFO
	Response 0:


```python
def reward_func(observations: np.ndarray, is_success: bool, is_failure: bool) -> float:
    x_velocity = observations[3]
    reward = x_velocity * 0.1

    if is_success:
        reward += 50.0
    elif is_failure:
        reward -= 20.0

    return reward
```


15:24:25 GenCode.py:228 DEBUG
	Reward function output: 0.0002014320699391295

15:24:25 PolicyTrainer.py:60 INFO
	state 1 begin is learning


Output()


15:28:53 PolicyTrainer.py:80 INFO
	state 1 has finished learning with performances: 0.0

15:28:53 PolicyTrainer.py:152 INFO
	the threshold is 0.9
/home/valentin/.conda/envs/llm/lib/python3.11/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/valentin/Travail/VIRAL/src/records/Swimmer-v5 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init
